<a href="https://colab.research.google.com/github/bpratham2001/Servilia/blob/main/notebooks/fine_tune_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nn = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
nn_fine_tuned = "Servilia-V1-8B-q8_0"
#nn = "deepseek-ai/DeepSeek-R1-Distill-Qwen-7B"

In [2]:
# @title Load data & imports
!pip install transformers
!pip install datasets
!pip install huggingface_hub
!pip install transformers
import pandas as pd
from random import randint as r
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import notebook_login
import torch
from torch.utils.data import DataLoader
import torch.distributed as dist
from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
from torch.distributed.fsdp import ShardingStrategy
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
#from google.colab import output

file =('wine-tastings-guide.txt')
df1 = pd.read_csv(file, sep = ' - ')
file =('FR-IT.txt')
df2 = pd.read_csv(file, sep = ' - ')
file =('grapes.txt')
df3 = pd.read_csv(file, sep = ' - ')
file =('wine-enthusiast.txt') # Questions already framed
df4 = pd.read_csv(file, sep = ' - ')
file =('essence-of-wine.txt') # Questions already framed
df5 = pd.read_csv(file, sep = ' - ')
#output.clear
print("total data points available: " + str(df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]))

total data points available: 275


<ipython-input-2-8f78077d9f77>:20: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df1 = pd.read_csv(file, sep = ' - ')
<ipython-input-2-8f78077d9f77>:22: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df2 = pd.read_csv(file, sep = ' - ')
<ipython-input-2-8f78077d9f77>:24: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df3 = pd.read_csv(file, sep = ' - ')
<ipython-input-2-8f78077d9f77>:26: ParserWarning: Falling bac

In [3]:
# @title Sample data

sample = pd.read_json("hf://datasets/yahma/alpaca-cleaned/alpaca_data_cleaned.json")

# prompts
tastes = ["Tell me more about wines that are", "Describe to me wines that feel",
          "I would like to know more about wines considered", "I want to know more about wines that are",
          "On wines, tell me the meaning of", "Explain, in wine terminology,", "Elaborate on wines that feel", "Enlighten me on wines that feel",
          "Educate me on wines that are", "I don't understand wines that are"]
tastes_q = ["What does it mean when a wine is", "How can a wine be", "How are wines considered", "How are wines"]
tastes_q2 = ["Why does", "How can", "Why would", "How does", "What does it mean if"]
tastes_q3 = ["feel ", "taste ", "seem "]
other = ["Tell me more about", "Describe", "I would like to know more about", "I want to know more about",
         "Tell me about", "Explain", "Elaborate on", "Enlighten me on", "Educate me on", "Recommend me ", "Give me a recommendation regarding "]
what_singular = ["What is", "What's"]

#unused
what_plural = ["What are"]
how_singular = ["How does", "How is", "How can", "How come"]
how_plural = ["How do", "How are", "How can", "How come"]
where_singular = ["Where is", "What is the location of", "Where can I find", "How do I come across"]
where_plural = ["Where are", "What are the locations of"]
why_plural = ["Why do", "Why are"]

# function to construct question/answers from blogs/articles
def construct_prompt(row, rng, q_type=0):
  if q_type == 0:#tastes
    if rng == 0:
      instruction = str(tastes[r(0, len(tastes)-1)] + " " +row['word']+".")
    elif rng == 1:
      instruction = str(tastes_q[r(0, len(tastes_q)-1)] + " " +row['word']+"?")
    else:
      instruction = str(tastes_q2[r(0, len(tastes_q2)-2)] + " a wine "+ tastes_q3[r(0,len(tastes_q3)-1)] +row['word']+"?")
  elif q_type == 1:#regions
    if rng == 0:
      instruction = str(other[r(0, len(other)-1)] + " " +row['word']+".")
    elif rng == 1:
      instruction = str("How is " +row['word']+"?")
    else:
      instruction = str(what_singular[r(0, len(what_singular)-1)] + " " +row['word']+" like?")
  else:#grapes
    if rng == 0:
      instruction = str(other[r(0, len(other)-1)] + " " +row['word']+" grapes.")
    elif rng == 1:
      instruction = str(what_singular[r(0, len(what_singular)-1)] + " " +row['word']+"?")
    else:
      instruction = str("How are the wines that are made from " +row['word']+" grapes?")
  output = row['description']
  return [instruction, output]

#sample.head()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
# @title Formatting data into alpaca format
df0 = pd.DataFrame(columns=['instruction', 'input', 'output'])

#construct_prompt(row, rng, q_type=0)
lst = [df1, df2, df3]
for i in range(len(lst)):
  for index,row in lst[i].iterrows():
    a = construct_prompt(row, r(0,2), i)
    df0.loc[len(df0)] = [a[0],'',a[1]]
for j in [df4, df5]:
  for index,row in j.iterrows():
    df0.loc[len(df0)] = [row['question'],'',row['answer']]
  if df0.shape[0] < (df1.shape[0]+df2.shape[0]+df3.shape[0]+df4.shape[0]+df5.shape[0]):
    df0 = df0.sample(frac=1)
df0 = df0.reset_index().drop('index', axis=1)
#df0.iloc[50:100]

In [ ]:
#@title Data prep

model = AutoModelForCausalLM.from_pretrained(nn)
tokenizer = AutoTokenizer.from_pretrained(nn)
tokenizer.pad_token = tokenizer.eos_token
dataset = Dataset.from_pandas(df0)

# Define a function to apply the chat template
def apply_chat_template(example):
    messages = [
        {"role": "user", "content": example['instruction']},
        {"role": "assistant", "content": example['output']}
    ]
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    return {"prompt": prompt}

# Apply the chat template function to the dataset
new_dataset = dataset.map(apply_chat_template)
new_dataset = new_dataset.train_test_split(0.05)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

# Tokenize the data
def tokenize_function(example):
    tokens = tokenizer(example['prompt'], padding="max_length", truncation=True, max_length=128)
    # Set padding token labels to -100 to ignore them in loss calculation
    tokens['labels'] = [
        -100 if token == tokenizer.pad_token_id else token for token in tokens['input_ids']
    ]
    return tokens

# Apply tokenize_function to each row
tokenized_train = train_dataset.map(tokenize_function)
tokenized_train = tokenized_train.remove_columns(['instruction', 'output', 'prompt'])
tokenized_test = test_dataset.map(tokenize_function)
tokenized_test = tokenized_test.remove_columns(['instruction', 'output', 'prompt'])

train_dataloader = DataLoader(tokenized_train, batch_size=2, shuffle=True)
test_dataloader = DataLoader(tokenized_test, batch_size=2)

config.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.67G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/7.39G [00:00<?, ?B/s]

In [ ]:
# @title FSDP

def setup_distributed():
    dist.init_process_group(backend="nccl")
def cleanup_distributed():
    dist.destroy_process_group()

setup_distributed()
model = FSDP(model)
model = FSDP(
    model,
    sharding_strategy=ShardingStrategy.FULL_SHARD,
    mixed_precision=True,
)

In [ ]:
# @title Training

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = CrossEntropyLoss()

def train(model, dataloader, optimizer, criterion, device):
    model.train()
    total_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)

num_epochs = 1
for epoch in range(num_epochs):
    train_loss = train(model, train_dataloader, optimizer, criterion, device)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_loss:.4f}")

cleanup_distributed()

In [ ]:
# @title Testing

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

            predictions = torch.argmax(outputs.logits, dim=-1)
            correct_predictions += (predictions == labels).sum().item()

    accuracy = correct_predictions / len(dataloader.dataset)
    return total_loss / len(dataloader), accuracy

test_loss, test_accuracy = evaluate(model, test_dataloader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# @title Saving

notebook_login()

!git clone https://huggingface.co/primetimepaper/DeepSeek-R1-Distills-Servilia
%cd DeepSeek-R1-Distills-Servilia

model.save_pretrained(nn_fine_tuned)

!git clone https://github.com/ggerganov/llama.cpp.git
!pip install -r llama.cpp/requirements.txt
!python llama.cpp/convert_hf_to_gguf.py {nn_fine_tuned}/  --outfile {nn_fine_tuned}.gguf  --outtype q8_0

!git lfs install
!git config --global http.version HTTP/1.1
#!git config http.postBuffer 8g
#!git config http.postBuffer 524288000
!git config --global core.compression 0
!git config --global user.email "bpratham2001@gmail.com"
!git config --global user.name "primetimepaper"

!huggingface-cli lfs-enable-largefiles .
!git lfs track "*.gguf"
#!git add .gitattributes
#!git commit -m "Add gitattributes for git lfs"
!git lfs migrate import --include="*.gguf"
!git add /content/DeepSeek-R1-Distills-Servilia/{nn_fine_tuned}
!git commit -m "bruh"

!git remote remove origin
!git remote add origin https://huggingface.co/primetimepaper/DeepSeek-R1-Distills-Servilia
!git push --set-upstream origin master
!git push